In [1]:
using Pkg
using DotEnv
include("src/test_name.jl")
using .Test_name

┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.to_rcm_time :: Tuple{String}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.bounding_box_to_geojson :: Tuple{}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.coordinate_to_geojson :: Union{Tuple{T}, Tuple{T, T}} where T<:Real` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.wkt_to_tuples :: Tuple{Any}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:243


In [2]:
DotEnv.config();

In [3]:

access_token = Test_name.Download.get_access_token_copernicus(username = get(ENV, "DATASPACE_USERNAME", ""), password = get(ENV, "DATASPACE_PASSWORD", ""));

In [4]:
data = Test_name.Download.Metadata.download_metadata_copernicus(bbox="4,51,4.5,52",start_datetime="2022-06-11T00:00:00Z", 
end_datetime="2022-06-22T23:59:59Z", max_records="100",collection="Sentinel1",product_type= "GRD",verbose=1);
df = Test_name.Download.Metadata.vector_of_dicts_to_dataframe(data["features"]);
println(df)

Request successful.


8×51 DataFrame
 Row │ properties_polarisation  properties_orbitDirection  properties_title                   type     properties_thumbnail               geometry_coordinates_1             properties_license_viewService  properties_license_licenseId  properties_cloudCover  id                                 properties_status  properties_swath  properties_links_1_rel  properties_centroid_type  properties_license_hasToBeSigned  properties_processingLevel  properties_gmlgeometry             properties_updated        properties_productType  properties_platform  properties_centroid_coordinates_2  properties_startDate      properties_license_signatureQuota  properties_license_description_shortName  properties_license_grantedOrganizationCountries  properties_parentIdentifier  properties_instrument  properties_processingBaseline  properties_description             properties_license_grantedFlags  properties_productIdentifier       properties_sensorMode  properties_published      properties_snow

In [ ]:
using HTTP
using ProgressMeter
using FileIO

function download_product2(access_token::String, product_id::String, filename::String)
    # URL of the file
    url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products($product_id)/\$value"

    # Authorization header
    headers = Dict("Authorization" => "Bearer $access_token")

    try
        # Make an HTTP GET request with the specified headers
        response = HTTP.get(url, headers=headers, request_timeout=30; stream=true)

        # Check if the response status is successful
        if HTTP.status(response) == 200
            # Get the content length from headers for progress meter
            content_length = parse(Int, get(response.headers, "Content-Length", "0"))

            # Create a progress meter
            p = Progress(content_length, desc="Downloading file")

            # Open the file for writing
            open(filename, "w") do file
                # Stream the response body and write to the file
                for chunk in response.body
                    write(file, chunk)
                    # Update the progress meter
                    ProgressMeter.update!(p, length(chunk))
                end
            end
            println("File downloaded successfully.")
        else
            println("Failed to download the file. Status code: ", HTTP.status(response))
        end
    catch e
        println("Error: ", e)
    end
end


In [15]:
download_product(access_token,df.id[2],"/Users/kaaso/Documents/coding/JuliaEO2024/data/raw/$(df.id[2]).zip")

In [7]:
Test_name.Download.Metadata.plot_polygons(df, "geometry_coordinates_1", "properties_updated")


UndefVarError: UndefVarError: Point not defined